In [ ]:
import pandas as pd

In [5]:
import pandas as pd
import datapane as dp
import altair as alt
import yfinance as yf

from scipy.stats import zscore

# Pull default parameters from our config file.
dp.Params.load_defaults('./datapane.yaml')

tickers = dp.Params.get('tickers')
plot_type = dp.Params.get('plot_type')
period = dp.Params.get('period')
calc_zscore = dp.Params.get('zscore')
data = yf.download(tickers=' '.join(tickers), period=period, groupby='ticker').Close

if calc_zscore:
    for ticker in tickers:
        data[ticker] = zscore(data[ticker])

df = data.reset_index().melt('Date', var_name='symbol', value_name='price')

base_chart = alt.Chart(df).encode(x='Date:T', y='price:Q', color='symbol').interactive().properties(width='container')

chart = base_chart.mark_line(point=True, size=5, opacity=0.5) if plot_type == 'line' else base_chart.mark_bar()
 
dp.Report(dp.Plot(chart), dp.Table(df)).publish(name=f'stock_report', headline=f'Report on {" ".join(tickers)}')

[*********************100%***********************]  3 of 3 completed


NameError: name 'chart' is not defined